In [78]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('Dataset/Bank.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp var rate: double (nullable = true)
 |-- cons price idx: double (nullable = true)
 |-- cons conf idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr employed: double (nullable = true)
 |-- Deposit: string (nullable = true)



In [93]:
# For integer type features, you may want to replace the null value the average of that feature.
# Let's see how that's done using balance as an example.
from pyspark.sql.functions import mean

# Let's find the average. But the problem is, how do we get that information out of the DataFrame?
mean_campaign = df.select(mean('campaign')).show()


+-----------------+
|    avg(campaign)|
+-----------------+
|2.567633847274493|
+-----------------+



In [94]:
df['campaign']=df['campaign'].fillna(df['campaign'].mean())

TypeError: 'Column' object is not callable

In [79]:
import pandas as pd

pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
age,56,57,37,40,56
job,housemaid,services,services,admin.,services
marital,married,married,married,married,married
education,basic.4y,high.school,high.school,basic.6y,high.school
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day,mon,mon,mon,mon,mon


In [80]:

df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp var rate',
 'cons price idx',
 'cons conf idx',
 'euribor3m',
 'nr employed',
 'Deposit']

In [81]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['age','duration','campaign','pdays','previous','emp var rate','cons price idx','cons conf idx','euribor3m','nr employed'], outputCol = 'features')

In [82]:
output = assembler.transform(df)

In [83]:

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = 'Deposit', outputCol = 'DepositIndex')
outputFixed = indexer.fit(output).transform(output)
outputFixed.printSchema()


root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp var rate: double (nullable = true)
 |-- cons price idx: double (nullable = true)
 |-- cons conf idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr employed: double (nullable = true)
 |-- Deposit: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- DepositIndex: double (nullable = true)



In [84]:
final_df = outputFixed.select('features', 'DepositIndex')
final_df.show(3)

+--------------------+------------+
|            features|DepositIndex|
+--------------------+------------+
|[56.0,261.0,1.0,9...|         0.0|
|[57.0,149.0,1.0,9...|         0.0|
|[37.0,226.0,1.0,9...|         0.0|
+--------------------+------------+
only showing top 3 rows



In [85]:
train, test = final_df.randomSplit([0.7, 0.3])

In [90]:
output.show


<bound method DataFrame.show of DataFrame[age: int, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, month: string, day: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, emp var rate: double, cons price idx: double, cons conf idx: double, euribor3m: double, nr employed: double, Deposit: string, features: vector]>

In [74]:
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, 
                                      GBTClassifier)
from pyspark.ml import Pipeline

dt = DecisionTreeClassifier(labelCol = 'DepositIndex', featuresCol = 'features')
rf = RandomForestClassifier(labelCol = 'DepositIndex', featuresCol = 'features')
gb = GBTClassifier(labelCol = 'DepositIndex', featuresCol = 'features')

In [75]:
dt_model = dt.fit(train)
rf_model = rf.fit(train)
gb_model = gb.fit(train)

Py4JJavaError: An error occurred while calling o1109.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 40.0 failed 1 times, most recent failure: Lost task 0.0 in stage 40.0 (TID 52, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<age_double_VectorAssembler_456781832f56d9e3e0df:double,duration_double_VectorAssembler_456781832f56d9e3e0df:double,campaign_double_VectorAssembler_456781832f56d9e3e0df:double,pdays_double_VectorAssembler_456781832f56d9e3e0df:double,previous_double_VectorAssembler_456781832f56d9e3e0df:double,emp var rate:double,cons price idx:double,cons conf idx:double,euribor3m:double,nr employed:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.sort_addToSorter$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.executi

In [17]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- nr employed: double (nullable = true)



In [21]:
train, test = final_data.randomSplit([0.7, 0.3])


In [42]:
lr = LinearRegression(labelCol = 'label')
lr_model = lr.fit(train)

IllegalArgumentException: 'Field "label" does not exist.'